# 1. Setup

In [1]:
# importing modules
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import fiona
import gdal

# setting up paths
path_languages = 'data/language_data/'
path_boundaries = 'data/boundaries/'
path_output = 'data/preprocessed/'

# setting up file names
fname_master = '1_Masterlist_161019.xlsx'
fname_stats = '11_BFS_IberischeSLW.xlsx'
fname_cantons = 'boundaries_swiss_cantons.shp'
fname_municipalities = 'swissBOUNDARIES3D_1_3_TLM_HOHEITSGEBIET.shp'

# 2. Loading data

Loading all language data (Bildung, Sprachschulen and Botschaften) and adding adding geometry columns 

In [8]:
sheet_names = ['Bildung','Sprachschulen','Botschaften']
dfs = [pd.read_excel(f'{path_languages}{fname_master}', sheet_name=name) for name in sheet_names]
df_bildung, df_sprachschulen, df_botschaften = dfs

In [9]:
df_bildung['geometry'] = df_bildung.apply(lambda x: Point(x['x '],x['y ']), axis=1)

df_bildung['ID']= df_bildung['ID'].astype(str)


# assign abbreviations for the different types of schools
dict_type_abbreviations = {
    'Agrupación de lengua y cultura española': 'ALCE',
    'Diploma de español lengua extranjera': 'DELE',
    'Instituto de ensenãnza media': 'IEM',
    'Universidad': 'U',
    'Centro universitário de idiomas': 'CUI',
    'Escuela de enseñanza primaria ': 'EEP'
}
df_bildung['type_abbr'] = df_bildung.apply(lambda x: dict_type_abbreviations[x['Typ']], axis=1)

gdf_bildung = gpd.GeoDataFrame(df_bildung, geometry='geometry')


# renaming columns
gdf_bildung = gdf_bildung.rename(columns={
    'Typ': 'typ',
    'Name': 'name',
    'Total': 'total'
})

# setting primary key
gdf_bildung = gdf_bildung.set_index('ID')

gdf_bildung.head()

,typ,name,x,y,total,geometry,type_abbr
ID,,,,,,,
#1,Agrupación de lengua y cultura española,ALCE,8.045701,47.390434,30,POINT (8.0457015 47.390434),ALCE
#2,Agrupación de lengua y cultura española,ALCE,7.600514,47.474485,38,POINT (7.600514 47.474485),ALCE
#3,Agrupación de lengua y cultura española,ALCE,8.309816,47.471368,71,POINT (8.309816 47.471368),ALCE
#4,Agrupación de lengua y cultura española,ALCE,7.605457,47.563638,66,POINT (7.605457 47.563638),ALCE
#5,Agrupación de lengua y cultura española,ALCE,7.588968,47.569927,67,POINT (7.588968 47.569927),ALCE


In [10]:
df_sprachschulen['geometry'] = df_sprachschulen.apply(lambda x: Point(x['x'],x['y']), axis=1)
gdf_sprachschulen = gpd.GeoDataFrame(df_sprachschulen, geometry='geometry')

# renaming columns
gdf_sprachschulen = gdf_sprachschulen.rename(columns={
    'Typ': 'typ',
    'Name': 'name',
    'Adresse': 'address'
})

# setting primary key
gdf_sprachschulen = gdf_sprachschulen.set_index('ID')

gdf_sprachschulen.head()

,typ,name,y,x,address,geometry
ID,,,,,,
#230,Escuela de idiomas,Klubschule Migros St. Gallen,47.425059,9.376588,"Marktgasse 17, 9000 St. Gallen",POINT (9.376587799999999 47.4250593)
#231,Escuela de idiomas,Klubschule Migros Arbon,47.514926,9.430403,"Rebenstrasse 20, 9320 Arbon",POINT (9.430402600000001 47.5149257)
#232,Escuela de idiomas,Klubschule Migros Buchs,7.164124,9.473690,"Churerstrasse 7, 9470 Buchs",POINT (9.47369 7.164124)
#233,Escuela de idiomas,Klubschule Migros Chur,46.852501,9.525666,"Gartenstrasse 5, 7001 Chur",POINT (9.525665999999999 46.852501)
#234,Escuela de idiomas,Klubschule Migros Frauenfeld,47.556236,8.896414,"Rheinstrasse 10, 8500 Frauenfeld",POINT (8.896413900000001 47.5562359)


In [11]:
# adding geometry column
df_botschaften['geometry'] = df_botschaften.apply(lambda x: Point(x['y'],x['x']), axis=1)
gdf_botschaften = gpd.GeoDataFrame(df_botschaften, geometry='geometry')


# renaming columns
gdf_botschaften = gdf_botschaften.rename(columns={
    'Länder': 'country',
    'Institution': 'institution',
    'laut EDA': 'name',
    'Adresse': 'address',
    'Unnamed: 5': 'website'
})

# adding column for leaflet marker (png file)
gdf_botschaften['marker'] = gdf_botschaften.apply(lambda x: f"{x['country'].lower()}_flag.png", axis=1)

# setting primary key
gdf_botschaften = gdf_botschaften.set_index('ID')
                                                  
gdf_botschaften.head()

,country,institution,name,address,website,x,y,geometry,marker
ID,,,,,,,,,
#345,Argentinien,Botschaft,Sección consular de la Embajada de la Republic...,"Jungfraustrasse 1, 3005 Berne",http://www.esuiz.mrecic.gob.ar,46.943993,7.454704,POINT (7.454704 46.943993),argentinien_flag.png
#346,Bolivia,Konsulat,Consulado général de la Republica de Bolivia,"Rue de Lausanne 72, 1202 Genève",http://www.Konsulatboliviasuiza.com,46.215722,6.147492,POINT (6.147492 46.215722),bolivia_flag.png
#347,Chile,Botschaft,Sección consular de la Embajada de Chile,"Eigerplatz 5, 3007 Berne",http://chileabroad.gov.cl/berna,46.940963,7.431628,POINT (7.431628 46.940963),chile_flag.png
#348,Chile,Konsulat,Consulado de la Republica de Chile,"Fuhrstrasse 12, Unterer Leihof, 8820 Wädenswil",NaN,47.225837,8.669301,POINT (8.669301000000001 47.225837),chile_flag.png
#349,Colombia,Konsulat,Sección consular de la Embajada de Colombia,"Zieglerstrasse 29, 3007 Berne",http://berna.Konsulat.gov.co,46.945014,7.429960,POINT (7.42996 46.945014),colombia_flag.png


Loading Swiss canton boundaries, fixing canton names, and adding centroid point to each canton

In [6]:
gdf_cantons = gpd.read_file(f'{path_boundaries}{fname_cantons}')
gdf_cantons = gdf_cantons[['NAME','geometry']]
gdf_cantons = gdf_cantons.rename(columns={'NAME': 'canton'})

gdf_cantons['boundary'] = gdf_cantons['geometry']

# adding centroid
gdf_cantons['geometry'] = gdf_cantons.apply(lambda x: Polygon(x['geometry']).centroid, axis=1)

# remapping names
def remap_names(row):
    dict_remap_names = {
        'Graub�nden': 'Graubünden',
        'Z�rich': 'Zürich',
        'Gen�ve': 'Genève',
        'Neuch�tel': 'Neuchâtel',
    }    
    old_name = row['canton']
    new_name = dict_remap_names.get(old_name,False)
    if not new_name:
        return old_name
    else:
        return new_name    
gdf_cantons['canton'] = gdf_cantons.apply(remap_names, axis=1)

gdf_cantons.head()

,canton,geometry,boundary
0,Graubï¿½nden,POINT (9.62862381268349 46.65606579160512),POLYGON Z ((8.877053154798531 46.8129134746790...
1,Bern,POINT (7.624744294648847 46.82208787558518),POLYGON Z ((7.153521643312189 46.9862818266617...
2,Valais,POINT (7.605940034344669 46.20935513994819),POLYGON Z ((8.47762548140741 46.52761948354787...
3,Vaud,POINT (6.646681769033475 46.55944971911612),POLYGON Z ((6.779825385513289 46.8529610472984...
4,Ticino,POINT (8.808554728958551 46.29606041574468),POLYGON Z ((8.47762548140741 46.52761948354787...


Loading Swiss municipality boundaries

In [15]:
gdf_municipalities = gpd.read_file(f'{path_boundaries}{fname_municipalities}')



"""
gdf_cantons = gdf_cantons[['NAME','geometry']]
gdf_cantons = gdf_cantons.rename(columns={'NAME': 'canton'})

gdf_cantons['boundary'] = gdf_cantons['geometry']

# adding centroid
gdf_cantons['geometry'] = gdf_cantons.apply(lambda x: Polygon(x['geometry']).centroid, axis=1)

# remapping names
def remap_names(row):
    dict_remap_names = {
        'Graub�nden': 'Graubünden',
        'Z�rich': 'Zürich',
        'Gen�ve': 'Genève',
        'Neuch�tel': 'Neuchâtel',
    }    
    old_name = row['canton']
    new_name = dict_remap_names.get(old_name,False)
    if not new_name:
        return old_name
    else:
        return new_name    
gdf_cantons['canton'] = gdf_cantons.apply(remap_names, axis=1)
"""


gdf_municipalities.head()

,UUID,DATUM_AEND,DATUM_ERST,ERSTELL_J,ERSTELL_M,REVISION_J,REVISION_M,GRUND_AEND,HERKUNFT,HERKUNFT_J,...,REVISION_Q,NAME,KANTONSNUM,ICC,EINWOHNERZ,BFS_NUMMER,GEM_TEIL,GEM_FLAECH,SHN,geometry
0,{D5A9677C-FE38-44F6-B0E8-3177F6252C81},2018-11-22,2008-11-24,2000,1,2019,1,Verbessert,AV,2019,...,2018_Aufbau,Scuol,18.0,CH,4598.0,3762,0,43861.0,CH18433762,POLYGON Z ((2827800.170000002 1186229.62000000...
1,{B55F9EAE-EB69-4468-A41C-7BB28A6699EE},2018-11-21,2008-11-24,2006,1,2019,1,Verbessert,AV,2019,...,2018_Aufbau,Glarus Süd,8.0,CH,9581.0,1631,0,43003.0,CH08001631,POLYGON Z ((2714000.934999999 1197614.67374999...
2,{AB2BA4A5-EB8C-40C7-A259-271DAC93C767},2018-11-21,2008-11-24,2000,1,2019,1,Verbessert,AV,2019,...,2018_Aufbau,Zernez,18.0,CH,1532.0,3746,0,34404.0,CH18433746,POLYGON Z ((2802711.668749999 1192062.36250000...
3,{550D0610-1EE3-4748-8F29-88072753FFDC},2018-11-21,2008-11-24,2000,1,2019,1,Verbessert,AV,2019,...,2018_Aufbau,Surses,18.0,CH,2343.0,3543,0,32377.0,CH18413543,POLYGON Z ((2759767.170000002 1151299.20625000...
4,{C9A12E0C-B661-4ED3-B5D2-C604EC55C7A6},2018-11-21,2008-11-24,1900,1,2019,1,Verbessert,AV,2019,...,2018_Aufbau,Lac Léman (VD),22.0,CH,NaN,9758,1,29756.0,CH22009758,POLYGON Z ((2549875.114999998 1147189.21999999...


Loading statistics

In [7]:
df_uni_stats = pd.read_excel(f'{path_languages}{fname_stats}', sheet_name='Studierende_new')
df_uni_stats['ID']= df_uni_stats['ID'].astype(str)

# renaming column names because geojson does not accept integers as column names
old_names = range(1988,2019)
new_names = [str(old_name) for old_name in old_names]
df_uni_stats = df_uni_stats.rename(columns=dict(zip(old_names, new_names)))


# setting primary key
df_uni_stats = df_uni_stats.set_index('ID')

df_uni_stats.head()

,name,1988,1989,1990,1991,1992,1993,1994,1995,1996,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
ID,,,,,,,,,,,,,,,,,,,,,
#191,Universität Basel,25,23,23,26,35,38,44,45,46,...,18,9,7,6,5,8,5,5,4,2
#192,Universität Bern,28,27,31,36,35,38,40,39,42,...,60,64,52,57,60,53,59,45,45,40
#193,Universität Frirbourg,16,19,20,19,21,23,30,24,24,...,73,76,70,69,68,60,57,60,60,53
#194,Universität Genf,80,82,80,81,94,87,90,89,84,...,39,41,31,24,31,30,32,30,32,24
#197,Université de Lausanne,17,20,18,21,26,32,38,41,43,...,21,24,22,23,24,22,17,10,8,9


# 3. Preprocessing

Adding canton to each point entry for all data frames

In [8]:
list_names = list(gdf_cantons['canton'])
list_boundaries = list(gdf_cantons['boundary'])

def in_canton(row):
    point = Point(row['geometry'])
    for name, boundary in zip(list_names, list_boundaries):
        if point.within(boundary):
            return name
    return 'not found'

gdf_bildung['canton'] = gdf_bildung.apply(in_canton, axis=1)
gdf_sprachschulen['canton'] = gdf_sprachschulen.apply(in_canton, axis=1)
gdf_botschaften['canton'] = gdf_botschaften.apply(in_canton, axis=1)

Getting total number of students (Bildung) per type for each canton

In [9]:
dict_total = gdf_bildung.groupby(['type_abbr','canton']).sum()['total'].to_dict()
def n_students(row):

    totals = [dict_total.get((type_abbr,row['canton']),0) for type_abbr in list(dict_type_abbreviations.values())]
    return totals

total_names_bildung = [f'total_{type_abbr}' for type_abbr in list(dict_type_abbreviations.values())]
gdf_cantons[total_names_bildung] = gdf_cantons.apply(n_students, axis=1, result_type='expand')
gdf_cantons.head()

,canton,geometry,boundary,total_ALCE,total_DELE,total_IEM,total_U,total_CUI,total_EEP
0,Graubï¿½nden,POINT (9.62862381268349 46.65606579160512),POLYGON Z ((8.877053154798531 46.8129134746790...,0,52,0,0,0,0
1,Bern,POINT (7.624744294648847 46.82208787558518),POLYGON Z ((7.153521643312189 46.9862818266617...,394,70,196,120,0,0
2,Valais,POINT (7.605940034344669 46.20935513994819),POLYGON Z ((8.47762548140741 46.52761948354787...,84,21,0,0,0,0
3,Vaud,POINT (6.646681769033475 46.55944971911612),POLYGON Z ((6.779825385513289 46.8529610472984...,612,43,902,302,401,0
4,Ticino,POINT (8.808554728958551 46.29606041574468),POLYGON Z ((8.47762548140741 46.52761948354787...,67,0,468,0,0,0


Getting total number of Sprachschulen

In [10]:
dict_schulen = gdf_sprachschulen.groupby('canton').size().to_dict()
print(dict_schulen)
def n_schulen(row):
    total = dict_schulen.get(row['canton'],0)
    return total

gdf_cantons['total_Schulen'] = gdf_cantons.apply(n_schulen, axis=1)
gdf_cantons.head()

{'Aargau': 9, 'Basel-Stadt': 2, 'Bern': 13, 'Fribourg': 3, 'Genï¿½ve': 10, 'Glarus': 1, 'Graubï¿½nden': 2, 'Luzern': 2, 'Neuchï¿½tel': 5, 'Schaffhausen': 1, 'Solothurn': 5, 'St. Gallen': 6, 'Thurgau': 3, 'Ticino': 11, 'Valais': 13, 'Vaud': 8, 'Zug': 3, 'Zï¿½rich': 17, 'not found': 1}


,canton,geometry,boundary,total_ALCE,total_DELE,total_IEM,total_U,total_CUI,total_EEP,total_Schulen
0,Graubï¿½nden,POINT (9.62862381268349 46.65606579160512),POLYGON Z ((8.877053154798531 46.8129134746790...,0,52,0,0,0,0,2
1,Bern,POINT (7.624744294648847 46.82208787558518),POLYGON Z ((7.153521643312189 46.9862818266617...,394,70,196,120,0,0,13
2,Valais,POINT (7.605940034344669 46.20935513994819),POLYGON Z ((8.47762548140741 46.52761948354787...,84,21,0,0,0,0,13
3,Vaud,POINT (6.646681769033475 46.55944971911612),POLYGON Z ((6.779825385513289 46.8529610472984...,612,43,902,302,401,0,8
4,Ticino,POINT (8.808554728958551 46.29606041574468),POLYGON Z ((8.47762548140741 46.52761948354787...,67,0,468,0,0,0,11


Getting total number of Botschaften and Konsulate for each canton

In [11]:
dict_institutions = gdf_botschaften.groupby(['institution','canton']).size().to_dict()
print(dict_institutions)

types_institutions = ['Botschaft','Konsulat']

def n_institutions(row):    
    totals = [dict_institutions.get((type_,row['canton']),0) for type_ in types_institutions]
    return totals


column_names = [f'total_{type_}' for type_ in types_institutions]
gdf_cantons[column_names] = gdf_cantons.apply(n_institutions, axis=1, result_type='expand')
gdf_cantons.head()

{('Botschaft', 'Bern'): 12, ('Botschaft', 'Genï¿½ve'): 4, ('Konsulat', 'Basel-Stadt'): 1, ('Konsulat', 'Bern'): 5, ('Konsulat', 'Genï¿½ve'): 5, ('Konsulat', 'Ticino'): 2, ('Konsulat', 'Vaud'): 1, ('Konsulat', 'Zï¿½rich'): 7, ('Konsulat', 'not found'): 1}


,canton,geometry,boundary,total_ALCE,total_DELE,total_IEM,total_U,total_CUI,total_EEP,total_Schulen,total_Botschaft,total_Konsulat
0,Graubï¿½nden,POINT (9.62862381268349 46.65606579160512),POLYGON Z ((8.877053154798531 46.8129134746790...,0,52,0,0,0,0,2,0,0
1,Bern,POINT (7.624744294648847 46.82208787558518),POLYGON Z ((7.153521643312189 46.9862818266617...,394,70,196,120,0,0,13,12,5
2,Valais,POINT (7.605940034344669 46.20935513994819),POLYGON Z ((8.47762548140741 46.52761948354787...,84,21,0,0,0,0,13,0,0
3,Vaud,POINT (6.646681769033475 46.55944971911612),POLYGON Z ((6.779825385513289 46.8529610472984...,612,43,902,302,401,0,8,0,1
4,Ticino,POINT (8.808554728958551 46.29606041574468),POLYGON Z ((8.47762548140741 46.52761948354787...,67,0,468,0,0,0,11,0,2


Creating data for university time series

In [12]:

gdf_unis_time_series = gdf_bildung.join(df_uni_stats, how='inner', lsuffix='_left', rsuffix='_right')
gdf_unis_time_series = gdf_unis_time_series.rename(columns={'name_left': 'name'})
gdf_unis_time_series = gdf_unis_time_series.drop(['name_right'], axis=1)


gdf_unis_time_series.head()


,typ,name,x,y,total,geometry,type_abbr,canton,1988,1989,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
ID,,,,,,,,,,,,,,,,,,,,,
#191,Universidad,Universität Basel,7.582858,47.558234,90,POINT (7.58285834556626 47.55823375),U,Basel-Stadt,25,23,...,18,9,7,6,5,8,5,5,4,2
#192,Universidad,Universität Bern,7.436919,46.950078,120,POINT (7.4369187 46.9500782),U,Bern,28,27,...,60,64,52,57,60,53,59,45,45,40
#193,Universidad,Universität Frirbourg,7.152665,46.806263,111,POINT (7.152665 46.8062633),U,Fribourg,16,19,...,73,76,70,69,68,60,57,60,60,53
#194,Universidad,Universität Genf,6.142629,46.199784,120,POINT (6.1426293 46.1997835),U,Genï¿½ve,80,82,...,39,41,31,24,31,30,32,30,32,24
#197,Universidad,Université de Lausanne,6.574077,46.528964,302,POINT (6.57407650067164 46.5289635),U,Vaud,17,20,...,21,24,22,23,24,22,17,10,8,9


# 4. Exporting

All files are exported as GeoJSON format

In [16]:

# canton data
keys = ['canton','geometry']


keys.extend(total_names_bildung)
gdf_cantons = gdf_cantons[keys]
gdf_cantons.to_file(f'{path_output}cantons.geojson', driver='GeoJSON')


# Bildung
gdf_bildung.to_file(f'{path_output}bildung.geojson', driver='GeoJSON')

# Sprachschulen
gdf_sprachschulen.to_file(f'{path_output}sprachschulen.geojson', driver='GeoJSON')

# Botschaften
gdf_botschaften.to_file(f'{path_output}botschaften.geojson', driver='GeoJSON')

# University stats
gdf_unis_time_series.to_file(f'{path_output}unis.geojson', driver='GeoJSON')

# Municipality data
gdf_municipalities.to_file(f'{path_output}municipalities.geojson', driver='GeoJSON')

# Preprocessing map 1

Loading and preprocessing immigration data

In [29]:
fname_immigration = 'Karte1_inmigración_total_nacionalidad.xlsx'

df_immigration = pd.read_excel(f'{path_languages}{fname_immigration}', sheet_name='Einwanderung_new')

df_immigration = df_immigration.rename(columns={'Nacionalidad': 'name'})

# used to retrieve flags from https://flag-icon-css.lip.is/
countryIDs = {
    'Argentinien': 'ar',
    'Bolivien': 'bo',
    'Chile': 'cl',
    'Colombia': 'co',
    'Costa Rica': 'cr',
    'Dominikanische Republik': 'do',
    'Ecuador': 'ec',
    'El Salvador': 'sv',
    'Guatemala': 'gt',
    'Honduras': 'hn',
    'Kolumbien': 'co',
    'Kuba': 'cu',
    'Mexiko': 'mx',
    'Nicaragua': 'ni',
    'Panama': 'pa',
    'Paraguay': 'py',
    'Peru': 'pe',
    'Spanien': 'es',
    'Uruguay': 'uy',
    'Venezuela': 've',  
};

# adding geometry column
df_immigration['code'] = df_immigration.apply(lambda row: countryIDs[row['name']], axis=1)


# setting primary key to country name
df_immigration = df_immigration.set_index('name')

df_immigration.head()

,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,code
name,,,,,,,,,,,,,,,,,,,,,
Argentinien,357,270,247,195,195,175,167,180,171,210,...,252,280,249,252,217,270,201,207,241,ar
Bolivien,95,79,80,76,86,81,71,91,97,89,...,252,185,148,135,117,134,156,167,167,bo
Chile,487,379,272,238,202,190,186,167,219,240,...,255,183,171,199,190,184,199,178,229,cl
Costa Rica,34,28,37,44,27,35,45,36,40,28,...,84,74,67,53,57,81,90,75,90,cr
Dominikanische Republik,275,555,694,659,741,631,562,565,511,548,...,416,392,392,447,336,393,341,325,281,do


Exporting as json file

In [31]:
df_immigration.to_json(f'../map1/map1.json', orient='table')

# Preprocessing map 2

Loading and preprocessing municipality data

In [58]:

# functions that returns a preprocessed data frame for an input year
def preprocess_stats_map2(year):
    
    fname_municipality_stats = 'Karte2_población_nacionalidad.xlsx'
    
    df_municipality_stats = pd.read_excel(f'{path_languages}{fname_municipality_stats}', sheet_name=f'{year}_new')

    # removing all non-municipality rows

    # function to determine whether a row is a municipality or not
    def is_municipality(row):
        place_name = row['Ortschaft']
        name_start = place_name.split()[0]

        # cantons start with -
        if name_start == '-':
            return False
        # districts start with >>
        elif name_start == '>>':
            return False
        else:
            return True

    df_municipality_stats['is_municipality'] = df_municipality_stats.apply(is_municipality,axis=1)    
    dropIndices = df_municipality_stats[df_municipality_stats['is_municipality'] == False].index
    df_municipality_stats = df_municipality_stats.drop(dropIndices)


    # adding bfs number column and municipality name column

    def split_column(row):
        place_name = row['Ortschaft']

        # splitting at first space
        bfs_number, name = place_name.split(' ', 1)
        bfs_number = int(bfs_number[-4:])

        return pd.Series({'bfs_number':bfs_number, 'name_old':name})

    new_columns = df_municipality_stats.apply(split_column, axis=1)                     
    df_municipality_stats = df_municipality_stats.merge(new_columns, left_index=True, right_index=True)                

    # put name column first
    df_municipality_stats.insert(0, 'name', df_municipality_stats.name_old)


    # removing Ortschaft column and is municipality column
    df_municipality_stats = df_municipality_stats.drop(labels=['Ortschaft', 'is_municipality', 'name_old'], axis=1)


    # setting primary key to bfs number
    df_municipality_stats = df_municipality_stats.set_index('bfs_number')


    # adding Total count
    countries = list(df_municipality_stats.columns)[1:]
    def sum_countries(row):
        total = 0
        for country in countries:
            total += row[country]
        return total

    df_municipality_stats['Total'] = df_municipality_stats.apply(sum_countries, axis=1)
    
    # renaming country counts according to year
    old_names = countries + ['Total']
    new_names = [f'{old_name}_{year}' for old_name in old_names]
    
    df_municipality_stats = df_municipality_stats.rename(columns=dict(zip(old_names, new_names)))
    
    return df_municipality_stats
    # end of the preprocessing function


    
startYear, endYear = 2010, 2018

df_municipality_stats = preprocess_stats_map2(startYear)

for year in range(startYear+1, endYear+1):
    df_year = preprocess_stats_map2(year)    
    df_year = df_year.drop(columns='name')
    df_municipality_stats = df_municipality_stats.join(df_year, how='inner', lsuffix='', rsuffix='')


df_municipality_stats.head()

,name,Spanien_2010,Argentinien_2010,Bolivien_2010,Costa Rica_2010,Dominikanische Republik_2010,Ecuador_2010,El Salvador_2010,Guatemala_2010,Honduras_2010,...,Kolumbien_2018,Kuba_2018,Mexiko_2018,Nicaragua_2018,Panama_2018,Paraguay_2018,Peru_2018,Uruguay_2018,Venezuela_2018,Total_2018
bfs_number,,,,,,,,,,,,,,,,,,,,,
1,Aeugst am Albis,4,1,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,15
2,Affoltern am Albis,39,2,3,0,12,2,0,0,0,...,9,2,3,0,0,1,1,0,0,101
3,Bonstetten,12,2,0,0,2,0,0,0,0,...,0,0,1,0,0,0,1,0,0,19
4,Hausen am Albis,3,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,5,13
5,Hedingen,11,3,0,0,0,0,0,0,0,...,2,0,1,0,0,0,0,0,1,32


Loading and preprocessing municipalities polygons

In [59]:
fname_municipalities = 'swissBOUNDARIES3D_1_3_TLM_HOHEITSGEBIET.shp'

gdf_municipalities = gpd.read_file(f'{path_boundaries}{fname_municipalities}')

# change crs to wg84
gdf_municipalities = gdf_municipalities.to_crs({'init' :'epsg:4326'})

# subset to Switzerland
gdf_municipalities = gdf_municipalities[gdf_municipalities['ICC']=='CH']

# subset to municipalities (Kantonsgebiet is removed)
gdf_municipalities = gdf_municipalities[gdf_municipalities['OBJEKTART']=='Gemeindegebiet']


gdf_municipalities = gdf_municipalities[['BFS_NUMMER', 'NAME', 'EINWOHNERZ', 'GEM_FLAECH', 'KANTONSNUM', 'geometry']]

rename_dict = {
    'BFS_NUMMER': 'bfs_number',
    'NAME': 'name',
    'EINWOHNERZ': 'n_inhabitants',
    'GEM_FLAECH': 'area',
    'KANTONSNUM': 'canton_number'
}

gdf_municipalities = gdf_municipalities.rename(columns=rename_dict)




# setting primary key to bfs number
gdf_municipalities = gdf_municipalities.set_index('bfs_number')

print(gdf_municipalities.shape)

gdf_municipalities.head()

(2307, 5)


,name,n_inhabitants,area,canton_number,geometry
bfs_number,,,,,
3762,Scuol,4598.0,43861.0,18.0,POLYGON Z ((10.42271965312055 46.7882429867244...
1631,Glarus Süd,9581.0,43003.0,8.0,POLYGON Z ((8.935383730092262 46.9198509326465...
3746,Zernez,1532.0,34404.0,18.0,POLYGON Z ((10.09692588743973 46.8487916580032...
3543,Surses,2343.0,32377.0,18.0,POLYGON Z ((9.519888855031379 46.4939395817611...
6031,Bagnes,8100.0,28410.0,23.0,POLYGON Z ((7.196372299565375 46.1140478064802...


Join municipality stats with boundaries

In [60]:
gdf_municipalities = gdf_municipalities.join(df_municipality_stats, how='left', lsuffix='_left', rsuffix='_right')
gdf_municipalities = gdf_municipalities.rename(columns={'name_left': 'name'})
gdf_municipalities = gdf_municipalities.drop(['name_right'], axis=1)

print(gdf_municipalities.shape)

gdf_municipalities.head()



(2307, 176)


,name,n_inhabitants,area,canton_number,geometry,Spanien_2010,Argentinien_2010,Bolivien_2010,Costa Rica_2010,Dominikanische Republik_2010,...,Kolumbien_2018,Kuba_2018,Mexiko_2018,Nicaragua_2018,Panama_2018,Paraguay_2018,Peru_2018,Uruguay_2018,Venezuela_2018,Total_2018
bfs_number,,,,,,,,,,,,,,,,,,,,,
1,Aeugst am Albis,1941.0,791.0,1.0,POLYGON Z ((8.502158500109211 47.2613186908263...,4.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0
2,Affoltern am Albis,12146.0,1059.0,1.0,POLYGON Z ((8.426138879874257 47.2791446160708...,39.0,2.0,3.0,0.0,12.0,...,9.0,2.0,3.0,0.0,0.0,1.0,1.0,0.0,0.0,101.0
3,Bonstetten,5512.0,743.0,1.0,POLYGON Z ((8.483766502017332 47.3237780927714...,12.0,2.0,0.0,0.0,2.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,19.0
4,Hausen am Albis,3664.0,1360.0,1.0,POLYGON Z ((8.574374122418318 47.2170667090754...,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,5.0,13.0
5,Hedingen,3694.0,653.0,1.0,POLYGON Z ((8.476753927364689 47.2930739975915...,11.0,3.0,0.0,0.0,0.0,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,32.0


Export as geojson

In [61]:
# export as geojson
gdf_municipalities.to_file(f'../map2/map2.geojson', driver='GeoJSON')

# Preprocessing map 3

Loading and preprocessing all bildung points

In [5]:
fname_bildung = '1_Masterlist_161019.xlsx'

df_bildung = pd.read_excel(f'{path_languages}{fname_bildung}', sheet_name='Bildung')


df_bildung['geometry'] = df_bildung.apply(lambda x: Point(x['x '],x['y ']), axis=1)

df_bildung['ID']= df_bildung['ID'].astype(str)


# assign abbreviations for the different types of schools
dict_type_abbreviations = {
    'Agrupación de lengua y cultura española': 'ALCE',
    'Diploma de español lengua extranjera': 'DELE',
    'Instituto de ensenãnza media': 'IEM',
    'Universidad': 'U',
    'Centro universitário de idiomas': 'CUI',
    'Escuela de enseñanza primaria ': 'EEP'
}
df_bildung['type_abbr'] = df_bildung.apply(lambda x: dict_type_abbreviations[x['Typ']], axis=1)

gdf_bildung = gpd.GeoDataFrame(df_bildung, geometry='geometry')


# renaming columns
gdf_bildung = gdf_bildung.rename(columns={
    'Typ': 'typ',
    'Name': 'name',
    'Total': 'total'
})

# setting primary key
gdf_bildung = gdf_bildung.set_index('ID')
gdf_unis = gdf_bildung[gdf_bildung['typ'] == 'Universidad']
print(gdf_unis.shape)


gdf_unis.head(100)

(9, 7)


,typ,name,x,y,total,geometry,type_abbr
ID,,,,,,,
#191,Universidad,Universität Basel,7.582858,47.558234,90,POINT (7.58285834556626 47.55823375),U
#192,Universidad,Universität Bern,7.436919,46.950078,120,POINT (7.4369187 46.9500782),U
#193,Universidad,Universität Frirbourg,7.152665,46.806263,111,POINT (7.152665 46.8062633),U
#194,Universidad,Universität Genf,6.142629,46.199784,120,POINT (6.1426293 46.1997835),U
#195,Universidad,Universität St. Gallen,9.374598,47.431768,173,POINT (9.37459796162333 47.43176765),U
#196,Universidad,Università della Svizzera italiana,9.374598,47.431768,130,POINT (9.37459796162333 47.43176765),U
#197,Universidad,Université de Lausanne,6.574077,46.528964,302,POINT (6.57407650067164 46.5289635),U
#198,Universidad,Universität Zürich,8.548334,47.374782,118,POINT (8.548333898386939 47.374782),U
#199,Universidad,CLC Centre for Languages and Communication (ZHAW),8.720451,47.499921,142,POINT (8.7204505 47.4999206),U


Loading and preprocessing statistics for universities

In [6]:
fname_stats = 'Karte3_Universitäten.xlsx'

df_uni_stats = pd.read_excel(f'{path_languages}{fname_stats}', sheet_name='studierende_unis')
df_uni_stats['ID']= df_uni_stats['ID'].astype(str)

# renaming column names because geojson does not accept integers as column names
old_names = range(1988,2019)
new_names = [str(old_name) for old_name in old_names]
df_uni_stats = df_uni_stats.rename(columns=dict(zip(old_names, new_names)))


# setting primary key
df_uni_stats = df_uni_stats.set_index('ID')

df_uni_stats.head(10)

,name,1988,1989,1990,1991,1992,1993,1994,1995,1996,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
ID,,,,,,,,,,,,,,,,,,,,,
#191,Universität Basel,25,23,23,26,35,38,44,45,46,...,18,9,7,6,5,8,5,5,4,2
#192,Universität Bern,28,27,31,36,35,38,40,39,42,...,60,64,52,57,60,53,59,45,45,40
#193,Universität Frirbourg,16,19,20,19,21,23,30,24,24,...,73,76,70,69,68,60,57,60,60,53
#194,Universität Genf,80,82,80,81,94,87,90,89,84,...,39,41,31,24,31,30,32,30,32,24
#197,Université de Lausanne,17,20,18,21,26,32,38,41,43,...,21,24,22,23,24,22,17,10,8,9
#198,Universität Zürich,101,100,108,96,104,105,86,67,85,...,119,124,112,107,109,100,83,71,54,49


Combining points with statistics

In [7]:
gdf_unis_time_series = gdf_bildung.join(df_uni_stats, how='inner', lsuffix='_left', rsuffix='_right')
gdf_unis_time_series = gdf_unis_time_series.rename(columns={'name_left': 'name'})
gdf_unis_time_series = gdf_unis_time_series.drop(['name_right'], axis=1)


gdf_unis_time_series.head()


,typ,name,x,y,total,geometry,type_abbr,1988,1989,1990,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
ID,,,,,,,,,,,,,,,,,,,,,
#191,Universidad,Universität Basel,7.582858,47.558234,90,POINT (7.58285834556626 47.55823375),U,25,23,23,...,18,9,7,6,5,8,5,5,4,2
#192,Universidad,Universität Bern,7.436919,46.950078,120,POINT (7.4369187 46.9500782),U,28,27,31,...,60,64,52,57,60,53,59,45,45,40
#193,Universidad,Universität Frirbourg,7.152665,46.806263,111,POINT (7.152665 46.8062633),U,16,19,20,...,73,76,70,69,68,60,57,60,60,53
#194,Universidad,Universität Genf,6.142629,46.199784,120,POINT (6.1426293 46.1997835),U,80,82,80,...,39,41,31,24,31,30,32,30,32,24
#197,Universidad,Université de Lausanne,6.574077,46.528964,302,POINT (6.57407650067164 46.5289635),U,17,20,18,...,21,24,22,23,24,22,17,10,8,9


Exporting as geojson

In [8]:
gdf_unis_time_series.to_file(f'../map3/map3.geojson', driver='GeoJSON', encoding='utf-8')

# Preprocessing map 3 chart

Loading and preprocessing Uni Stufe data

In [15]:
fname_stats = 'Karte3_Universitäten.xlsx'

df_uni_stats = pd.read_excel(f'{path_languages}{fname_stats}', sheet_name='studierende_stufe')

# renaming column names because geojson does not accept integers as column names
old_names = range(1988,2019)
new_names = [str(old_name) for old_name in old_names]
df_uni_stats = df_uni_stats.rename(columns=dict(zip(old_names, new_names)))

print(list(df_uni_stats['type']))

dict_type_abbreviations = {
    'Total': 't',
    'Lizenziat / Diplom': 'ld',
    'Bachelor': 'b',
    'Master': 'm',
    'Doktorat': 'd',
    'Weiterbildung, Vertiefung und andere': 'wva'
}
df_uni_stats['code'] = df_uni_stats.apply(lambda x: dict_type_abbreviations[x['type']], axis=1)


df_uni_stats.head(10)

['Total', 'Lizenziat / Diplom', 'Bachelor', 'Master', 'Doktorat', 'Weiterbildung, Vertiefung und andere']


,type,1988,1989,1990,1991,1992,1993,1994,1995,1996,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,code
0,Total,288,291,306,304,350,353,362,334,365,...,357,307,299,307,286,268,237,218,192,t
1,Lizenziat / Diplom,243,245,255,258,301,316,335,311,328,...,81,54,43,35,30,14,8,5,12,ld
2,Bachelor,0,0,0,0,0,0,0,0,0,...,143,115,127,114,104,107,91,66,50,b
3,Master,0,0,0,0,0,0,0,0,0,...,70,80,80,101,89,85,74,90,72,m
4,Doktorat,28,30,34,30,35,28,23,19,27,...,58,49,42,54,54,54,57,55,50,d
5,"Weiterbildung, Vertiefung und andere",17,16,17,16,14,9,4,4,10,...,5,9,7,3,9,8,7,2,8,wva


Exporting as json

In [16]:
df_uni_stats.to_json(f'../map3/map3_chart.json', orient='table')

# Preprocessing map 5

Loading and preprocessing Sprachschulen data

In [40]:
fname_schools = 'Karte5_Sprachschulen_Nov.xlsx'

df_schools = pd.read_excel(f'{path_languages}{fname_schools}', sheet_name='Tabelle1')

df_schools = df_schools.rename(columns={
    'Typ': 'type',
    'Name': 'name',
    'Adresse': 'address',
    'Webseite': 'website'
})

# setting primary key to country name
df_schools = df_schools.set_index('ID')

# converting to geodataframe
df_schools['geometry'] = df_schools.apply(lambda x: Point(x['x'],x['y']), axis=1)
gdf_schools = gpd.GeoDataFrame(df_schools, geometry='geometry')

# getting rid of unnecessary columns
gdf_schools = gdf_schools.drop(columns=['x', 'y', 'Unnamed: 7'])

df_schools.head()

,type,name,y,x,address,website,Unnamed: 7,geometry
ID,,,,,,,,
#230,Escuela de idiomas,Klubschule Migros St. Gallen,47.425059,9.376588,"Marktgasse 17, 9000 St. Gallen",https://www.klubschule.ch/Standorte/Ostschweiz...,NaN,POINT (9.376587799999999 47.4250593)
#231,Escuela de idiomas,Klubschule Migros Arbon,47.514926,9.430403,"Rebenstrasse 20, 9320 Arbon",https://www.klubschule.ch/Standorte/Ostschweiz...,NaN,POINT (9.430402600000001 47.5149257)
#232,Escuela de idiomas,Klubschule Migros Buchs,7.164124,9.473690,"Churerstrasse 7, 9470 Buchs",https://www.klubschule.ch/Standorte/Ostschweiz...,NaN,POINT (9.47369 7.164124)
#233,Escuela de idiomas,Klubschule Migros Chur,46.852501,9.525666,"Gartenstrasse 5, 7001 Chur",https://www.klubschule.ch/Standorte/Ostschweiz...,NaN,POINT (9.525665999999999 46.852501)
#234,Escuela de idiomas,Klubschule Migros Frauenfeld,47.556236,8.896414,"Rheinstrasse 10, 8500 Frauenfeld",https://www.klubschule.ch/Standorte/Ostschweiz...,NaN,POINT (8.896413900000001 47.5562359)


Export as geojson

In [43]:
gdf_schools.to_file(f'../map5/map5.geojson', driver='GeoJSON', encoding='utf-8')

# Preprocessing map 6

Loading embassy data and preprocessing

In [4]:
fname_embassy ='1_Masterlist_161019.xlsx'
df_embassy = pd.read_excel(f'{path_languages}{fname_embassy}', sheet_name='Botschaften')

# adding geometry column
# df_embassy['geometry'] = df_embassy.apply(lambda x: Point(x['y'],x['x']), axis=1)
# gdf_embassy = gpd.GeoDataFrame(df_embassy, geometry='geometry')
# gdf_embassy = gdf_embassy.drop(columns=['x', 'y'])

# renaming columns
df_embassy = df_embassy.rename(columns={
    'Länder': 'country',
    'Institution': 'institution',
    'laut EDA': 'name',
    'Adresse': 'address',
    'Unnamed: 5': 'website',
    'x': 'y',
    'y': 'x',
})

# used to retrieve flags from https://flag-icon-css.lip.is/
countryIDs = {
    'Argentinien': 'ar',
    'Bolivia': 'bo',
    'Chile': 'cl',
    'Colombia': 'co',
    'Costa Rica': 'cr',
    'Cuba': 'cu',
    'Ecuador': 'ec',
    'El Salvador': 'sv',
    'España': 'es',
    'Guatemala': 'gt',
    'Honduras': 'hn',
    'México': 'mx',
    'Nicaragua': 'ni',
    'Pánama': 'pa',
    'Paraguay': 'py',
    'Peru': 'pe',
    'Republica Dominicana': 'do',
    'Uruguay': 'uy',
    'Venezuela': 've',
};
# adding country codes
df_embassy['code'] = df_embassy.apply(lambda row: countryIDs[row['country']], axis=1)


# setting primary key
df_embassy = df_embassy.set_index('ID')


df_embassy['geometry'] = df_embassy.apply(lambda x: Point(x['x'],x['y']), axis=1)
gdf_embassy = gpd.GeoDataFrame(df_embassy, geometry='geometry')

gdf_embassy = gdf_embassy.drop(columns=['x', 'y'])


gdf_embassy.head()

,country,institution,name,address,website,code,geometry
ID,,,,,,,
#345,Argentinien,Botschaft,Sección consular de la Embajada de la Republic...,"Jungfraustrasse 1, 3005 Berne",http://www.esuiz.mrecic.gob.ar,ar,POINT (7.454704 46.943993)
#346,Bolivia,Konsulat,Consulado général de la Republica de Bolivia,"Rue de Lausanne 72, 1202 Genève",http://www.Konsulatboliviasuiza.com,bo,POINT (6.147492 46.215722)
#347,Chile,Botschaft,Sección consular de la Embajada de Chile,"Eigerplatz 5, 3007 Berne",http://chileabroad.gov.cl/berna,cl,POINT (7.431628 46.940963)
#348,Chile,Konsulat,Consulado de la Republica de Chile,"Fuhrstrasse 12, Unterer Leihof, 8820 Wädenswil",NaN,cl,POINT (8.669301000000001 47.225837)
#349,Colombia,Konsulat,Sección consular de la Embajada de Colombia,"Zieglerstrasse 29, 3007 Berne",http://berna.Konsulat.gov.co,co,POINT (7.42996 46.945014)


Saving embassy to geojson

In [6]:
gdf_embassy.to_file(f'../map6/map6.geojson', driver='GeoJSON', encoding='utf-8')

# df_embassy.to_json(path_or_buf= f'../map6/map6.json', orient='records')

# gdf_embassy.to_file(f'../map6/map6.geojson', driver='GeoJSON')

# Preprocessing map 7

Loading Kantis data and preprocessing

In [27]:
fname_kantis = 'Karte7_Kantis_Total.xlsx'
df_kantis = pd.read_excel(f'{path_languages}{fname_kantis}', sheet_name='Tabelle1')

df_kantis['geometry'] = df_kantis.apply(lambda x: Point(x['y'],x['x']), axis=1)

gdf_kantis = gpd.GeoDataFrame(df_kantis, geometry='geometry')


# renaming columns
gdf_kantis = gdf_kantis.rename(columns={
    'Name': 'name',
    'Adresse': 'address'
})


gdf_kantis = gdf_kantis[['name', 'address', 'geometry']]

print(gdf_kantis.shape)
gdf_kantis.head()



(108, 3)


,name,address,geometry
0,Academia Engiadina Samedan Mittelschule,"Quadratscha 18, 7503 Samedan",POINT (6.658441 46.506575)
1,Alte Kantonsschule Aarau,"Bahnhofstrasse 91, 5001 Aarau",POINT (8.053175 47.393547)
2,Bündner Kantonsschule,"Arosastrasse 2, 7000 Chur",POINT (9.536315999999999 46.848527)
3,Collège Calvin,"Rue Théodore De-Bèze 2-4, 1206 Genève",POINT (6.152066 46.200454)
4,Collège Claparède,"Chemin de Fossard 61, 1231 Chêne-Bougeries",POINT (6.184946 46.18937)


Loading and preprocessing canton borders

In [28]:
fname_cantons = 'boundaries_swiss_cantons.shp'
gdf_cantons = gpd.read_file(f'{path_boundaries}{fname_cantons}')
print(gdf_cantons.columns)
gdf_cantons = gdf_cantons[['NAME', 'KT_TEIL', 'EINWOHNERZ', 'geometry']]
gdf_cantons = gdf_cantons.rename(columns={'NAME': 'name'})

gdf_cantons['geometry']


# remapping names
def remap_names(row):
    dict_remap_names = {
        'Graubï¿½nden': 'Graubünden',
        'Zï¿½rich': 'Zürich',
        'Genï¿½ve': 'Genève',
        'Neuchï¿½tel': 'Neuchâtel',
    }    
    old_name = row['name']
    new_name = dict_remap_names.get(old_name,False)
    if not new_name:
        return old_name
    else:
        return new_name    
gdf_cantons['name'] = gdf_cantons.apply(remap_names, axis=1)

gdf_cantons.head()

Index(['UUID', 'DATUM_AEND', 'DATUM_ERST', 'ERSTELL_J', 'ERSTELL_M',
       'REVISION_J', 'REVISION_M', 'GRUND_AEND', 'HERKUNFT', 'HERKUNFT_J',
       'HERKUNFT_M', 'OBJEKTART', 'REVISION_Q', 'ICC', 'KANTONSNUM',
       'SEE_FLAECH', 'KANTONSFLA', 'KT_TEIL', 'NAME', 'EINWOHNERZ',
       'geometry'],
      dtype='object')


,name,KT_TEIL,EINWOHNERZ,geometry
0,Graubünden,0,197888.0,POLYGON Z ((8.877053154798531 46.8129134746790...
1,Bern,1,1031126.0,POLYGON Z ((7.153521643312189 46.9862818266617...
2,Valais,0,341463.0,POLYGON Z ((8.47762548140741 46.52761948354787...
3,Vaud,1,793129.0,POLYGON Z ((6.779825385513289 46.8529610472984...
4,Ticino,0,353709.0,POLYGON Z ((8.47762548140741 46.52761948354787...


Assigning canton to each kanti

In [29]:


list_names = list(gdf_cantons['name'])
list_boundaries = list(gdf_cantons['geometry'])

def in_canton(row):
    point = Point(row['geometry'])
    for name, boundary in zip(list_names, list_boundaries):
        if point.within(boundary):
            return name
    return 'not found'

gdf_kantis['canton'] = gdf_kantis.apply(in_canton, axis=1)



gdf_kantis.head()





,name,address,geometry,canton
0,Academia Engiadina Samedan Mittelschule,"Quadratscha 18, 7503 Samedan",POINT (6.658441 46.506575),Vaud
1,Alte Kantonsschule Aarau,"Bahnhofstrasse 91, 5001 Aarau",POINT (8.053175 47.393547),Aargau
2,Bündner Kantonsschule,"Arosastrasse 2, 7000 Chur",POINT (9.536315999999999 46.848527),Graubünden
3,Collège Calvin,"Rue Théodore De-Bèze 2-4, 1206 Genève",POINT (6.152066 46.200454),Genève
4,Collège Claparède,"Chemin de Fossard 61, 1231 Chêne-Bougeries",POINT (6.184946 46.18937),Genève


Export kantis data as geojson

In [35]:
gdf_kantis.to_file(f'../map7/map7_kantis.geojson', driver='GeoJSON', encoding='utf-8')

Get number of kantis for each canton

In [30]:
gdf_kantis['count'] = 1
dict_n_kantis = gdf_kantis.groupby(['canton']).sum()['count'].to_dict()

print(dict_n_kantis)

def n_kantis(row):
    return dict_n_kantis.get(row['name'], 0)

gdf_cantons['n_kantis'] = gdf_cantons.apply(n_kantis, axis=1, result_type='expand')
gdf_cantons.head()


{'Aargau': 6, 'Appenzell Ausserrhoden': 2, 'Appenzell Innerrhoden': 1, 'Basel-Landschaft': 6, 'Basel-Stadt': 4, 'Bern': 9, 'Fribourg': 7, 'Genève': 10, 'Glarus': 1, 'Graubünden': 4, 'Jura': 2, 'Luzern': 6, 'Neuchâtel': 2, 'Nidwalden': 1, 'Schaffhausen': 1, 'Schwyz': 3, 'Solothurn': 1, 'St. Gallen': 5, 'Thurgau': 3, 'Ticino': 5, 'Uri': 2, 'Vaud': 13, 'Zug': 5, 'Zürich': 8, 'not found': 1}


,name,KT_TEIL,EINWOHNERZ,geometry,n_kantis
0,Graubünden,0,197888.0,POLYGON Z ((8.877053154798531 46.8129134746790...,4
1,Bern,1,1031126.0,POLYGON Z ((7.153521643312189 46.9862818266617...,9
2,Valais,0,341463.0,POLYGON Z ((8.47762548140741 46.52761948354787...,0
3,Vaud,1,793129.0,POLYGON Z ((6.779825385513289 46.8529610472984...,13
4,Ticino,0,353709.0,POLYGON Z ((8.47762548140741 46.52761948354787...,5


Export data for choropleth map (kantis per inhabitants)

In [31]:
gdf_cantons_choropleth = gdf_cantons.drop(columns=['KT_TEIL'])
gdf_cantons_choropleth = gpd.GeoDataFrame(gdf_cantons_choropleth, geometry='geometry')

gdf_cantons_choropleth.to_file(f'../map7/map7_choropleth.geojson', driver='GeoJSON', encoding='utf-8')

Subset cantons such that there is only 1 entry (polygon) per canton.
In case of multiple polygons, the largest one is used.

In [32]:
gdf_cantons = gdf_cantons[(gdf_cantons['KT_TEIL'] == '0') | (gdf_cantons['KT_TEIL'] == '1')]

print(gdf_cantons.shape)
gdf_cantons.head(26)

(26, 5)


,name,KT_TEIL,EINWOHNERZ,geometry,n_kantis
0,Graubünden,0,197888.0,POLYGON Z ((8.877053154798531 46.8129134746790...,4
1,Bern,1,1031126.0,POLYGON Z ((7.153521643312189 46.9862818266617...,9
2,Valais,0,341463.0,POLYGON Z ((8.47762548140741 46.52761948354787...,0
3,Vaud,1,793129.0,POLYGON Z ((6.779825385513289 46.8529610472984...,13
4,Ticino,0,353709.0,POLYGON Z ((8.47762548140741 46.52761948354787...,5
5,St. Gallen,1,504686.0,POLYGON Z ((8.808608711153505 47.2200878229021...,5
6,Zürich,0,1504346.0,POLYGON Z ((8.410083928295883 47.2483745726766...,8
7,Fribourg,1,315074.0,POLYGON Z ((7.040344112497663 46.9795230264076...,7
8,Luzern,0,406506.0,POLYGON Z ((8.46816705302245 46.99652349168549...,6
9,Aargau,1,670988.0,POLYGON Z ((8.410083928295883 47.2483745726766...,6


Compute centroid for each polygon (centroid will correspond to the marker position on the map)

In [33]:
# adding centroid
gdf_cantons['geometry'] = gdf_cantons.apply(lambda x: Polygon(x['geometry']).centroid, axis=1)
gdf_cantons.head()

,name,KT_TEIL,EINWOHNERZ,geometry,n_kantis
0,Graubünden,0,197888.0,POINT (9.62862381268349 46.65606579160512),4
1,Bern,1,1031126.0,POINT (7.624744294648847 46.82208787558518),9
2,Valais,0,341463.0,POINT (7.605940034344669 46.20935513994819),0
3,Vaud,1,793129.0,POINT (6.646681769033475 46.55944971911612),13
4,Ticino,0,353709.0,POINT (8.808554728958551 46.29606041574468),5


Exporting as geojson

In [34]:
gdf_cantons = gdf_cantons.drop(columns=['KT_TEIL', 'EINWOHNERZ'])
gdf_cantons = gpd.GeoDataFrame(gdf_cantons, geometry='geometry')
gdf_cantons.to_file(f'../map7/map7_absolut.geojson', driver='GeoJSON', encoding='utf-8')

# Preprocessing map 8

Loading all language data (Bildung, Sprachschulen and Botschaften) and adding adding geometry columns 

In [15]:
sheet_names = ['Bildung','Sprachschulen','Botschaften']
dfs = [pd.read_excel(f'{path_languages}{fname_master}', sheet_name=name) for name in sheet_names]
df_bildung, df_sprachschulen, df_botschaften = dfs

df_bildung['geometry'] = df_bildung.apply(lambda x: Point(x['x '],x['y ']), axis=1)

df_bildung['ID']= df_bildung['ID'].astype(str)


# assign abbreviations for the different types of schools
dict_type_abbreviations = {
    'Agrupación de lengua y cultura española': 'ALCE',
    'Diploma de español lengua extranjera': 'DELE',
    'Instituto de ensenãnza media': 'IEM',
    'Universidad': 'U',
    'Centro universitário de idiomas': 'CUI',
    'Escuela de enseñanza primaria ': 'EEP'
}
df_bildung['type_abbr'] = df_bildung.apply(lambda x: dict_type_abbreviations[x['Typ']], axis=1)

gdf_bildung = gpd.GeoDataFrame(df_bildung, geometry='geometry')


# renaming columns
gdf_bildung = gdf_bildung.rename(columns={
    'Typ': 'typ',
    'Name': 'name',
    'Total': 'total'
})

# setting primary key
gdf_bildung = gdf_bildung.set_index('ID')

gdf_bildung.head()


,typ,name,x,y,total,geometry,type_abbr
ID,,,,,,,
#1,Agrupación de lengua y cultura española,ALCE,8.045701,47.390434,30,POINT (8.0457015 47.390434),ALCE
#2,Agrupación de lengua y cultura española,ALCE,7.600514,47.474485,38,POINT (7.600514 47.474485),ALCE
#3,Agrupación de lengua y cultura española,ALCE,8.309816,47.471368,71,POINT (8.309816 47.471368),ALCE
#4,Agrupación de lengua y cultura española,ALCE,7.605457,47.563638,66,POINT (7.605457 47.563638),ALCE
#5,Agrupación de lengua y cultura española,ALCE,7.588968,47.569927,67,POINT (7.588968 47.569927),ALCE


Saving as geojson

In [13]:
gdf_bildung.to_file(f'../map8/map8.geojson', driver='GeoJSON', encoding='utf-8')